In [1]:
import re
import nltk
import nltk.data

from bs4 import BeautifulSoup
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer

import pandas as pd
import numpy as np

In [2]:
df_temp = pd.read_csv('training.1600000.processed.noemoticon.csv', index_col= None, encoding = "ISO-8859-1", 
                      names=['value','views'], usecols=[5,0])

In [13]:
from sklearn.utils import shuffle
df_temp = shuffle(df_temp)

temp_X = df_temp["views"]
temp_y = df_temp['value']

from sklearn.model_selection import train_test_split

X_train, X_cv, y_train, y_cv = train_test_split(temp_X, temp_y, test_size=0.4)
print(X_train.shape[0])
print(X_cv.shape[0])

960000
640000


In [14]:
def review_to_word_list(review, remove_stopwords=False, use_stemmer=False, use_lemmatizer=False):
    # 1. Remove HTML. First, we'll remove the HTML tags. For this purpose, 
    # we'll use the Beautiful Soup
    review_text = BeautifulSoup(review,"lxml").get_text()
    
    # 2. Remove non-letters
    review_text = re.sub("[^a-zA-Z]", " ", review_text)
    
    # 3. Convert words to lower case and split them
    words = review_text.lower().split()
    # 4. Optionally remove stop words (false by default)
    if remove_stopwords:
        stops = set(stopwords.words("english"))
        words = [w for w in words if not w in stops]
        
    # 5. Optionally use Porter Stemmer
    if use_stemmer:
        porter_stemmer = PorterStemmer()
        words = [porter_stemmer.stem(w) for w in words]
    
    # 6. Optionally use Lemmatizer
    if use_lemmatizer:
        wordnet_lemmatizer = WordNetLemmatizer()
        words = [wordnet_lemmatizer.lemmatize(w) for w in words]
    #
    # 7. Return a list of words
    return words

In [15]:
def review_to_sentences(review, tokenizer, remove_stopwords=False, use_stemmer=False, use_lemmatizer=False):
    # Function to split a review into parsed sentences. Returns a
    # list of sentences, where each sentence is a list of words
    #
    # 1. Use the NLTK tokenizer to split the paragraph into sentences
    # raw_sentences = tokenizer.tokenize(review.decode('utf8').strip())
    raw_sentences = tokenizer.tokenize(review.strip())

    # 2. Loop over each sentence
    sentences = []
    for raw_sentence in raw_sentences:
        # If a sentence is empty, skip it
        if len(raw_sentence) > 0:
            # Otherwise, call review_to_wordlist to get a list of words
            sentences.append(review_to_word_list(raw_sentence,remove_stopwords,use_stemmer, use_lemmatizer))

    # Return the list of sentences (each sentence is a list of words,
    # so this returns a list of lists
    return sentences

In [16]:
# Load the punkt tokenizer
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')

In [17]:
import warnings
warnings.filterwarnings('ignore')

sentences = []  # Initialize an empty list of sentences

train_X_val = X_train.values
i=0
num_reviews  = X_train.size
print ("Parsing sentences from training set")
for view in train_X_val:
    i = i+1
    if ((i + 1) % 160000 == 0):
        print ("Review %d of %d" % (i + 1, num_reviews))
    sentences += review_to_sentences(view, tokenizer)

print('Complete')

Parsing sentences from training set
Review 160000 of 960000
Review 320000 of 960000
Review 480000 of 960000
Review 640000 of 960000
Review 800000 of 960000
Review 960000 of 960000
Complete


In [18]:
print (len(sentences))

1640841


In [19]:
print (sentences[0])

['just', 'watched', 'one', 'tree', 'hill', 'episode', 'season', 'never', 'cried', 'this', 'much', 'since', 'keith', 'died']


In [21]:
# Import the built-in logging module and configure it so that Word2Vec creates nice output messages
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s',level=logging.INFO)

# Set values for various parameters
num_features = 400    # Word vector dimensionality                      
min_word_count = 40   # Minimum word count                        
num_workers = 6       # Number of threads to run in parallel
context = 10          # Context window size                                                                                    
downsampling = 1e-3   # Downsample setting for frequent words

In [22]:
# Initialize and train the model (this will take some time)
from gensim.models import word2vec
print ("Training model...")
model = word2vec.Word2Vec(sentences, workers=num_workers, size=num_features, 
                          min_count = min_word_count, window = context, sample = downsampling)

# If you don't plan to train the model any further, calling 
# init_sims will make the model much more memory-efficient.
model.init_sims(replace=True)

# It can be helpful to create a meaningful model name and 
# save the model for later use. You can load it later using Word2Vec.load()
model_name = "400features_40minwords_10context"
model.save(model_name)

print('Completed Training')

2017-01-05 14:37:16,555 : INFO : 'pattern' package not found; tag filters are not available for English
2017-01-05 14:37:16,561 : INFO : collecting all words and their counts
2017-01-05 14:37:16,561 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2017-01-05 14:37:16,588 : INFO : PROGRESS: at sentence #10000, processed 80133 words, keeping 12537 word types
2017-01-05 14:37:16,617 : INFO : PROGRESS: at sentence #20000, processed 159331 words, keeping 20020 word types
2017-01-05 14:37:16,646 : INFO : PROGRESS: at sentence #30000, processed 240159 words, keeping 26518 word types
2017-01-05 14:37:16,674 : INFO : PROGRESS: at sentence #40000, processed 319914 words, keeping 32343 word types
2017-01-05 14:37:16,705 : INFO : PROGRESS: at sentence #50000, processed 400163 words, keeping 37733 word types
2017-01-05 14:37:16,733 : INFO : PROGRESS: at sentence #60000, processed 479904 words, keeping 42877 word types
2017-01-05 14:37:16,764 : INFO : PROGRESS: at sentence 

Training model...


2017-01-05 14:37:16,796 : INFO : PROGRESS: at sentence #80000, processed 640157 words, keeping 52362 word types
2017-01-05 14:37:16,826 : INFO : PROGRESS: at sentence #90000, processed 719798 words, keeping 56856 word types
2017-01-05 14:37:16,855 : INFO : PROGRESS: at sentence #100000, processed 800416 words, keeping 61264 word types
2017-01-05 14:37:16,885 : INFO : PROGRESS: at sentence #110000, processed 879938 words, keeping 65368 word types
2017-01-05 14:37:16,915 : INFO : PROGRESS: at sentence #120000, processed 959393 words, keeping 69510 word types
2017-01-05 14:37:16,945 : INFO : PROGRESS: at sentence #130000, processed 1038395 words, keeping 73454 word types
2017-01-05 14:37:16,974 : INFO : PROGRESS: at sentence #140000, processed 1117411 words, keeping 77404 word types
2017-01-05 14:37:17,004 : INFO : PROGRESS: at sentence #150000, processed 1196341 words, keeping 81266 word types
2017-01-05 14:37:17,034 : INFO : PROGRESS: at sentence #160000, processed 1276050 words, keepin

Completed Training


In [23]:
model.doesnt_match("man woman child kitchen".split())

'kitchen'

In [25]:
model.most_similar("man")

[('guy', 0.547832727432251),
 ('boy', 0.5378686189651489),
 ('dude', 0.5372099876403809),
 ('woman', 0.5245792865753174),
 ('girl', 0.4487118721008301),
 ('gosh', 0.44019967317581177),
 ('kid', 0.438829243183136),
 ('nigga', 0.4201546013355255),
 ('bastard', 0.4112083911895752),
 ('chick', 0.4029228091239929)]

In [30]:
# Load the module
from gensim.models import Word2Vec
model = Word2Vec.load("400features_40minwords_10context")

2017-01-05 14:48:58,192 : INFO : loading Word2Vec object from 400features_40minwords_10context
2017-01-05 14:48:58,396 : INFO : setting ignored attribute cum_table to None
2017-01-05 14:48:58,397 : INFO : setting ignored attribute syn0norm to None
2017-01-05 14:48:58,398 : INFO : loaded 400features_40minwords_10context


In [33]:
type(model.syn0)

numpy.ndarray

In [34]:
def makeFeatureVec(words, model, num_features):
    # Function to average all of the word vectors in a given paragraph
    
    # Pre-initialize an empty numpy array (for speed)
    featureVec = np.zeros((num_features,),dtype="float32")
    nwords = 0.
    
    # Index2word is a list that contains the names of the words in 
    # the model's vocabulary. Convert it to a set, for speed 
    index2word_set = set(model.index2word)
    
    # Loop over each word in the review and, if it is in the model's
    # vocaublary, add its feature vector to the total
    for word in words:
        if word in index2word_set: 
            nwords = nwords + 1.
            featureVec = np.add(featureVec,model[word])
    
    # Divide the result by the number of words to get the average
    featureVec = np.divide(featureVec,nwords)
    return featureVec

In [37]:
def getAvgFeatureVecs(views, model, num_features):
    # Given a set of reviews (each one a list of words), calculate 
    # the average feature vector for each one and return a 2D numpy array
    
    # Initialize a counter
    counter = 0.
    
    # Preallocate a 2D numpy array, for speed
    viewFeatureVecs = np.zeros((len(views),num_features),dtype="float32")
    
    # Loop through the reviews
    for view in views:
        # Print a status message every 5000th review
        if counter%30000. == 0.:
           print ("Review %d of %d" % (counter, len(views)))
        # Call the function (defined above) that makes average feature vectors
        viewFeatureVecs[counter] = makeFeatureVec(view, model, num_features)
        counter = counter + 1.
    
    return viewFeatureVecs

In [38]:
# Calculate average feature vectors for training and testing sets, using the functions we defined above. 
# Notice that we now use stop word removal.
print ("Creating average feature vecs for train views")
clean_train_views = []
for view in train_X_val:
    clean_train_views.append( review_to_word_list( view,remove_stopwords=True ))
    
trainDataVecs = getAvgFeatureVecs( clean_train_views, model, num_features )

print('Complete average feature vector')

Creating average feature vecs for train views
Review 0 of 960000
Review 30000 of 960000
Review 60000 of 960000
Review 90000 of 960000
Review 120000 of 960000
Review 150000 of 960000
Review 180000 of 960000
Review 210000 of 960000
Review 240000 of 960000
Review 270000 of 960000
Review 300000 of 960000
Review 330000 of 960000
Review 360000 of 960000
Review 390000 of 960000
Review 420000 of 960000
Review 450000 of 960000
Review 480000 of 960000
Review 510000 of 960000
Review 540000 of 960000
Review 570000 of 960000
Review 600000 of 960000
Review 630000 of 960000
Review 660000 of 960000
Review 690000 of 960000
Review 720000 of 960000
Review 750000 of 960000
Review 780000 of 960000
Review 810000 of 960000
Review 840000 of 960000
Review 870000 of 960000
Review 900000 of 960000
Review 930000 of 960000
Complete average feature vector


In [43]:
bad_indices = np.where(np.isnan(trainDataVecs))
trainDataVecs[bad_indices] = 0
# print(bad_indices)

In [44]:
print("Training the random forest...")
from sklearn.ensemble import RandomForestClassifier

# Initialize a Random Forest classifier with 100 trees
forest = RandomForestClassifier(n_estimators = 100, n_jobs = 6) # number of cores

# Fit the forest to the training set, using the bag of words as 
# features and the sentiment labels as the response variable
#
# This may take a few minutes to run
forest = forest.fit(trainDataVecs, y_train.values)

print('Training complete.')

Training the random forest...
Training complete.


In [45]:
df_test = pd.read_csv('testdata.manual.2009.06.14.csv', index_col= None, encoding = "ISO-8859-1", 
                      names=['value','views'], usecols=[5,0])

In [46]:
print(df_test.head())

   value                                              views
0      4  @stellargirl I loooooooovvvvvveee my Kindle2. ...
1      4  Reading my kindle2...  Love it... Lee childs i...
2      4  Ok, first assesment of the #kindle2 ...it fuck...
3      4  @kenburbary You'll love your Kindle2. I've had...
4      4  @mikefish  Fair enough. But i have the Kindle2...


In [49]:
test_X = df_test["views"]
test_y = df_test['value']
print(test_X.shape[0])

498


In [50]:
print ("Creating average feature vecs for test reviews")
X_test_val = test_X.values
clean_test_reviews = []
for review in X_test_val:
    clean_test_reviews.append( review_to_word_list( review, remove_stopwords=True ))

testDataVecs = getAvgFeatureVecs( clean_test_reviews, model, num_features )

print('Completing feature creation')

Creating average feature vecs for test reviews
Review 0 of 498
Completing feature creation


In [52]:
bad_indices = np.where(np.isnan(testDataVecs))
testDataVecs[bad_indices] = 0

In [53]:
from sklearn.model_selection import cross_val_score
test_scores = cross_val_score(forest,testDataVecs, test_y.values)
print(test_scores)

[ 0.62275449  0.6626506   0.63030303]
